# Analysis of EBA Transparency Exercise Datasets

## Introduction

This notebook explores the datasets provided by the European Banking Authority (EBA) under their EU-wide Transparency Exercise, accessible [here](https://www.eba.europa.eu/risk-analysis-and-data/eu-wide-transparency-exercise). These datasets offer a comprehensive view of the banking sector's assets and risks, which is invaluable for financial analysis and regulatory assessment.

### Notes

- **Understand the Structure**: We will examine the structure and content of the EBA's credit risk dataset to understand the type of data provided and its potential applications.
- **Ensure Data Integrity**: Although these datasets are generally user-friendly, they require additional checks to validate data integrity and reliability. You should compare a sample of your intermediary results with the results presented by the EBA's interactive tool, specifically the visualizations found [here](https://tools.eba.europa.eu/interactive-tools/2023/powerbi/837203/tr23_visualisation_page5.html), to confirm consistency and understand any discrepancies.
- **Download the credit risk datasets from here**: https://www.eba.europa.eu/assets/TE2023/Full_database/837203/tr_cre.csv

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

pd.options.display.float_format = '{:,.2f}'.format

#### A. Create base table

In [25]:
# Import mappning tables
exposures = pd.read_excel('metadata_tr_2023.xlsx', sheet_name='Exposures')
portfolio = pd.read_excel('metadata_tr_2023.xlsx', sheet_name='Portfolio')
fin_instrument = pd.read_excel('metadata_tr_2023.xlsx', sheet_name='Financial_Instruments')
assets_stages = pd.read_excel('metadata_tr_2023.xlsx', sheet_name='ASSETS_Stages')
assets_fv = pd.read_excel('metadata_tr_2023.xlsx', sheet_name='ASSETS_FV')
institutions = pd.read_excel('metadata_tr_2023.xlsx', sheet_name='List of institutions')

# import core dataset
df = pd.read_csv('tr_oth_2023.csv', low_memory=False)

In [26]:
# Create final dataset
result = pd.merge(df, exposures, on='Exposure', how='left')
result = pd.merge(result, fin_instrument, on='Financial_instruments', how='left')
result = pd.merge(result, assets_stages, on='ASSETS_Stages', how='left')
result = pd.merge(result, assets_fv, on='ASSETS_FV', how='left')
result = pd.merge(result, institutions, on='LEI_Code', how='left')

result.head()

,LEI_Code,NSA,Period,Item,Label,ASSETS_FV,ASSETS_Stages,Exposure,Financial_instruments,Amount,Fin_end_year,n_quarters,Footnote,Row,Column,Sheet,exposure_label,Financial_instruments_label,ASSETS_Stages_label,ASSETS_FV_label,institution_country,Desc_country,Name,Finrep,Fin_year_end
0,0W2PZJM8XOY22M4GG883,DE,202209,2321001,"Cash, cash balances at central banks and other demand deposits",0,0,0,0,"22,438.22",0,3,NaN,10,5,Assets,Total / No breakdown,Total / No breakdown,No breakdown by ASSETS_Stages,No breakdown by ASSETS_FV,DE,Germany,DekaBank Deutsche Girozentrale,Yes - IFRS,31/12
1,0W2PZJM8XOY22M4GG883,DE,202209,2321002,Financial assets held for trading,0,0,0,0,"16,279.12",0,3,NaN,11,5,Assets,Total / No breakdown,Total / No breakdown,No breakdown by ASSETS_Stages,No breakdown by ASSETS_FV,DE,Germany,DekaBank Deutsche Girozentrale,Yes - IFRS,31/12
2,0W2PZJM8XOY22M4GG883,DE,202209,2321002,Financial assets held for trading,1,0,0,0,"3,956.74",0,3,NaN,11,6,Assets,Total / No breakdown,Total / No breakdown,No breakdown by ASSETS_Stages,Fair value hierarchy: Level 1,DE,Germany,DekaBank Deutsche Girozentrale,Yes - IFRS,31/12
3,0W2PZJM8XOY22M4GG883,DE,202209,2321002,Financial assets held for trading,2,0,0,0,"11,736.86",0,3,NaN,11,7,Assets,Total / No breakdown,Total / No breakdown,No breakdown by ASSETS_Stages,Fair value hierarchy: Level 2,DE,Germany,DekaBank Deutsche Girozentrale,Yes - IFRS,31/12
4,0W2PZJM8XOY22M4GG883,DE,202209,2321002,Financial assets held for trading,3,0,0,0,585.52,0,3,NaN,11,8,Assets,Total / No breakdown,Total / No breakdown,No breakdown by ASSETS_Stages,Fair value hierarchy: Level 3,DE,Germany,DekaBank Deutsche Girozentrale,Yes - IFRS,31/12


In [27]:
result['Label'].unique()

array(['Cash, cash balances at central banks and other demand deposits',
       'Financial assets held for trading',
       'Non-trading financial assets mandatorily at fair value through profit or loss',
       'Financial assets designated at fair value through profit or loss',
       'Financial assets at fair value through other comprehensive income',
       'Financial assets at amortised cost',
       'Derivatives – Hedge accounting',
       'Fair value changes of the hedged items in portfolio hedge of interest rate risk',
       'Other assets', 'Total Assets',
       'Gross carrying amount: Financial assets at fair value through other comprehensive income, Debt securities',
       'Gross carrying amount: Financial assets at fair value through other comprehensive income, Loans and advances',
       'Gross carrying amount: Financial assets at amortised cost, Debt securities',
       'Gross carrying amount: Financial assets at amortised cost, Loans and advances',
       'Accumulated i

### B. Compare stage 2 to stage 3

In [33]:
# Create base table
idx_1 = result['Label'].isin(['Accumulated impairment: Financial assets at amortised cost, Loans and advances'])
idx_2 = result['Period'] == 202306

pivot_df = pd.pivot_table(result[idx_1 & idx_2], 
                          values='Amount', 
                          index=['Name'], 
                          columns=['ASSETS_Stages'], 
                          aggfunc="sum", 
                          margins=True,
                          fill_value=0).reset_index()

pivot_df.sort_values(by=pivot_df.columns[-1], ascending=True).head(40)

ASSETS_Stages,Name,1,2,3,All
112,All,"-31,895.95","-54,005.76","-148,668.11","-234,569.82"
21,"Banco Santander, S.A.","-3,941.48","-5,224.76","-14,130.64","-23,296.88"
53,Groupe Crédit Agricole,"-2,986.88","-5,806.15","-12,265.42","-21,058.45"
15,BNP Paribas,"-2,050.02","-2,476.04","-13,272.16","-17,798.22"
52,Groupe BPCE,"-1,310.86","-3,993.91","-8,717.38","-14,022.15"
39,Confédération Nationale du Crédit Mutuel,"-1,920.20","-2,124.87","-7,309.19","-11,354.26"
19,"Banco Bilbao Vizcaya Argentaria, S.A.","-2,143.18","-2,021.14","-7,106.52","-11,270.84"
106,"UNICREDIT, SOCIETA' PER AZIONI","-1,396.87","-3,856.85","-5,792.77","-11,046.49"
99,Société générale S.A.,"-1,052.14","-2,036.60","-7,578.85","-10,667.59"
61,Intesa Sanpaolo S.p.A.,-780.01,"-1,671.22","-4,973.36","-7,424.59"


In [38]:
# Create base table
idx_1 = result['Label'].isin(['Gross carrying amount: Financial assets at amortised cost, Loans and advances'])
idx_2 = result['Period'] == 202306

pivot_df = pd.pivot_table(result[idx_1 & idx_2], 
                          values='Amount', 
                          index=['Name'], 
                          columns=['ASSETS_Stages'], 
                          aggfunc="sum", 
                          margins=True,
                          fill_value=0).reset_index()

pivot_df.columns = pivot_df.columns.astype(str)
pivot_df['Stage_1_vs_Stage_2'] = pivot_df['2'] / pivot_df['1']

pivot_df.sort_values(by='All', ascending=False).head(40)

ASSETS_Stages,Name,1,2,3,All,Stage_1_vs_Stage_2
112,All,"13,788,690.65","1,411,158.98","333,903.79","15,533,753.42",0.10
53,Groupe Crédit Agricole,"1,138,802.13","111,480.18","25,153.91","1,275,436.22",0.10
21,"Banco Santander, S.A.","1,009,877.95","72,759.93","33,575.03","1,116,212.91",0.07
52,Groupe BPCE,"795,888.56","126,951.26","21,024.95","943,864.78",0.16
15,BNP Paribas,"831,092.50","70,675.02","25,926.28","927,693.80",0.09
39,Confédération Nationale du Crédit Mutuel,"645,056.14","40,543.74","15,388.53","700,988.41",0.06
59,ING Groep N.V.,"621,071.11","49,906.81","10,648.75","681,626.67",0.08
42,DEUTSCHE BANK AKTIENGESELLSCHAFT,"530,274.20","52,086.53","12,146.44","594,507.18",0.10
99,Société générale S.A.,"491,919.26","36,930.96","16,436.19","545,286.41",0.08
106,"UNICREDIT, SOCIETA' PER AZIONI","410,441.67","79,767.72","12,163.08","502,372.48",0.19


### C. Compare IRB Shortfall

In [51]:
# Create base table
idx_1 = result['Period'] == 202306
idx_2 = result['Label'].isin(['(-) IRB shortfall of credit risk adjustments to expected losses'])
idx_3 = result['Label'].isin(['Accumulated impairment: Financial assets at amortised cost, Loans and advances'])
idx_4 = result['ASSETS_Stages'] = 3

pivot_df = pd.pivot_table(result[idx_1 & (idx_2 | (idx_3 & idx_4))], 
                          values='Amount', 
                          index=['Name'], 
                          columns=['Label'], 
                          aggfunc="sum", 
                          # margins=True,
                          fill_value=0).reset_index()

pivot_df.columns = pivot_df.columns.astype(str)
pivot_df['irb_shortfall_vs_stage_3_provisions'] = pivot_df['(-) IRB shortfall of credit risk adjustments to expected losses'] / pivot_df['Accumulated impairment: Financial assets at amortised cost, Loans and advances']

pivot_df.sort_values(by='Accumulated impairment: Financial assets at amortised cost, Loans and advances', ascending=True).head(40)

Label,Name,(-) IRB shortfall of credit risk adjustments to expected losses,"Accumulated impairment: Financial assets at amortised cost, Loans and advances",irb_shortfall_vs_stage_3_provisions
21,"Banco Santander, S.A.",-250.09,"-23,296.88",0.01
60,Groupe Crédit Agricole,-371.25,"-21,058.45",0.02
15,BNP Paribas,-478.67,"-17,798.22",0.03
59,Groupe BPCE,-191.32,"-14,022.15",0.01
44,Confédération Nationale du Crédit Mutuel,-381.31,"-11,354.26",0.03
19,"Banco Bilbao Vizcaya Argentaria, S.A.",-193.96,"-11,270.84",0.02
116,"UNICREDIT, SOCIETA' PER AZIONI",-6.75,"-11,046.49",0.00
109,Société générale S.A.,-359.58,"-10,667.59",0.03
68,Intesa Sanpaolo S.p.A.,-230.80,"-7,424.59",0.03
40,"CaixaBank, S.A.",0.00,"-7,151.07",-0.00
